# Spotify Data Analysis Project
The goal of this project is to analyze user data that can be obtained through Spotify after a long time using the platform.
The analysis of personal data is now a main selling point for spotify with their "spotify's wrapped" release every year for each user.
Being able to analyze the data without waiting a year for it can be greatly insightful and fun to do!

### Imports

In [2]:
#!/usr/bin/python3

import pandas as pd
from dotenv import load_dotenv
from os import getenv

In [75]:
load_dotenv("./spotify.env")

# import all streamin history values
strm0=getenv("streaming0")
print(strm0)
strm1=getenv("streaming1")
print(strm1)
strm2=getenv("streaming2")
print(strm2)

./Spotify Account Data/StreamingHistory_music_0.json
./Spotify Account Data/StreamingHistory_music_1.json
./Spotify Account Data/StreamingHistory_music_2.json


In [76]:
stream_db0 = pd.read_json(strm0)
stream_db1 = pd.read_json(strm1)
stream_db2= pd.read_json(strm2)

temp = pd.concat([stream_db0,stream_db1],ignore_index=True)
streaming_history = pd.concat([temp, stream_db2],ignore_index=True)


print(streaming_history.head(10))
print(streaming_history.size)

            endTime          artistName  \
0  2023-03-13 12:52          Madvillain   
1  2023-03-15 11:24         Jean Dawson   
2  2023-03-19 23:17     Superstar Pride   
3  2023-03-20 09:29          Dreamville   
4  2023-03-20 09:33      Kendrick Lamar   
5  2023-03-20 09:38               Drake   
6  2023-03-20 09:42          Kanye West   
7  2023-03-20 09:43                Joji   
8  2023-03-20 09:47         Joey Bada$$   
9  2023-03-20 09:50  Tyler, The Creator   

                                           trackName  msPlayed  
0                                       Meat Grinder     73231  
1                                         BAD FRUIT*      4080  
2                                  PAINTING PICTURES     43073  
3  Sacrifices (with EARTHGANG & J. Cole feat. Smi...    382306  
4                                             PRIDE.    275253  
5                                           Too Much    261866  
6                                            Bound 2    229146  
7     

### Preparing the data

In [77]:
print(streaming_history.sort_values(by="endTime",ascending=False).iloc[0:1])
print(streaming_history.sort_values(by="endTime",ascending=True).iloc[0:1])

                endTime artistName trackName  msPlayed
26559  2024-03-20 18:03       MAVI     Sense       980
            endTime  artistName     trackName  msPlayed
0  2023-03-13 12:52  Madvillain  Meat Grinder     73231


Analysis conducted between 2023-03-13 12:52 and 2024-03-20 18:03

In [78]:
streaming_history.drop(columns="endTime",inplace=True) #removing the endTime column

In [79]:
streaming_history["msPlayed"] = streaming_history["msPlayed"]/1000 #time played in seconds
streaming_history.rename(columns={"msPlayed":"TimePlayedS"},inplace=True) #renaming the columnn
streaming_history.head(5)

,artistName,trackName,TimePlayedS
0,Madvillain,Meat Grinder,73.231
1,Jean Dawson,BAD FRUIT*,4.080
2,Superstar Pride,PAINTING PICTURES,43.073
3,Dreamville,Sacrifices (with EARTHGANG & J. Cole feat. Smi...,382.306
4,Kendrick Lamar,PRIDE.,275.253


Now creating new columns for the time played, one in minutes and one in hours.

In [80]:
streaming_history["TimePlayedM"]=streaming_history["TimePlayedS"]/60
streaming_history["TimePlayedH"]=streaming_history["TimePlayedM"]/60

In [90]:
streaming_history[streaming_history["TimePlayedM"]<=1]#index


,artistName,trackName,TimePlayedS,TimePlayedM,TimePlayedH
1,Jean Dawson,BAD FRUIT*,4.080,0.068000,0.001133
2,Superstar Pride,PAINTING PICTURES,43.073,0.717883,0.011965
14,SZA,Doves In The Wind (feat. Kendrick Lamar),0.023,0.000383,0.000006
15,Kanye West,Everything I Am,1.812,0.030200,0.000503
16,MF DOOM,Doomsday,0.160,0.002667,0.000044
...,...,...,...,...,...
26555,Thee Sacred Souls,Can I Call You Rose?,1.570,0.026167,0.000436
26556,SZA,Saturn - Live,1.270,0.021167,0.000353
26557,Flo Milli,Never Lose Me (feat. SZA & Cardi B),54.880,0.914667,0.015244
26558,Chuck Strangers,Backwood Falls,1.180,0.019667,0.000328


In [91]:
streaming_history.drop(streaming_history[streaming_history["TimePlayedM"]<=1].index, inplace=True)
streaming_history.dropna(inplace=True)

Only keeping the songs that have been listened to for at least a minute.

### Analyzing the data

In [118]:
streaming_history.groupby(by="trackName").agg({"artistName":"first","TimePlayedS":"sum","TimePlayedM":"sum","TimePlayedH":"sum"}).sort_values(by='TimePlayedS',ascending=False)

,artistName,TimePlayedS,TimePlayedM,TimePlayedH
trackName,,,,
MY EYES,Travis Scott,20567.336,342.788933,5.713149
Fallin',Joey Bada$$,15034.098,250.568300,4.176138
Be Your Girl (Kaytranada Edition),Teedra Moses,12349.202,205.820033,3.430334
Hummingbird (Metro Boomin & James Blake),Metro Boomin,9766.528,162.775467,2.712924
PRIDE.,Kendrick Lamar,8825.884,147.098067,2.451634
...,...,...,...,...
The Middle of the World,Nicholas Britell,60.388,1.006467,0.016774
PLACE ON FIRE,Jasiah,60.267,1.004450,0.016741
Currents,Drake,60.200,1.003333,0.016722


In [107]:
streaming_history[streaming_history["artistName"].str.contains("Travis")].groupby(by="trackName").sum().sort_values(by="TimePlayedS",ascending=False)


,artistName,TimePlayedS,TimePlayedM,TimePlayedH
trackName,,,,
MY EYES,Travis ScottTravis ScottTravis ScottTravis Sco...,20567.336,342.788933,5.713149
MELTDOWN (feat. Drake),Travis ScottTravis ScottTravis ScottTravis Sco...,8819.836,146.997267,2.449954
GOD'S COUNTRY,Travis ScottTravis ScottTravis ScottTravis Sco...,5822.634,97.043900,1.617398
FE!N (feat. Playboi Carti),Travis ScottTravis ScottTravis ScottTravis Sco...,5129.403,85.490050,1.424834
SIRENS,Travis ScottTravis ScottTravis ScottTravis Sco...,4428.951,73.815850,1.230264
TOPIA TWINS (feat. Rob49 & 21 Savage),Travis ScottTravis ScottTravis ScottTravis Sco...,4327.058,72.117633,1.201961
LOST FOREVER (feat. Westside Gunn),Travis ScottTravis ScottTravis ScottTravis Sco...,4224.968,70.416133,1.173602
TIL FURTHER NOTICE (feat. James Blake & 21 Savage),Travis ScottTravis ScottTravis ScottTravis Sco...,4030.382,67.173033,1.119551
DELRESTO (ECHOES) (feat. Beyoncé),Travis ScottTravis ScottTravis ScottTravis Sco...,3931.441,65.524017,1.092067
